In [1]:
# Import libraries

import csv
import re
import time
import os
import sys

from tqdm import tqdm
from tqdm.notebook import trange, tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib.parse import urljoin

from threading import Thread

import pandas as pd

In [2]:
base_path = os.path.join(os.path.abspath(''), 'data')

#driver = webdriver.Chrome()
driver = webdriver.Firefox()
driver.get('https://www.dnn.de/suche/?query=Russland')

- Put in your Filter if you want now
- Open some articles first to get rid of unwanted popups

In [3]:
# Webscraping algorithm

iterations = 556

regex = r"^Kostenpflichtig"

not_readable = []

hits = 11105

df = pd.DataFrame(columns=['title','author','category', 'date', 'text'])
df.index.name = 'row_id'

pbar = tqdm(total=hits, position=0, leave=True)
for i in range(iterations):

    driver.switch_to.window(driver.window_handles[0])

    try:
        wait = WebDriverWait(driver, 5) 
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div#resultdata > a')))
        links = driver.find_elements(By.CSS_SELECTOR, 'div#resultdata > a')
    except:
        print("no links on page: ", i)
        continue

    for j in range(len(links)):
        pbar.update(1)
    
        driver.switch_to.window(driver.window_handles[0])

        links[j].send_keys(Keys.CONTROL + Keys.RETURN)
        driver.switch_to.window(driver.window_handles[-1])

        try:      

            wait = WebDriverWait(driver, 5) 
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'header h2')))           

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            title = soup.select_one('article header > div > h2').text

            re_list = re.findall(regex, title)
            if len(re_list) > 0:
                not_readable.append(driver.current_url)
                driver.close()
                
                continue
            
            resort = str(driver.current_url).split('/')[3]

            date = soup.select_one('article header time').text
            
            text = soup.select('article > div > div > p')
            text = [t.text for t in text]
            text = " ".join(text)

            author = soup.select_one('article header a[rel="author"]')
            if author is not None:
                author = author.text
            else:
                author = 'NA'
        except:
            print("Error loading page: ", driver.current_url)
            driver.close()

            continue

        driver.close()

        df2 = pd.DataFrame({'title': title, 'author': author,'category':resort, 'date': date, 'text': text},index=[0])
        df2 = df2.astype(str)
        
        df = pd.concat([df, df2], ignore_index=True)
        
        time.sleep(0.75) # To not overload the server (hopefully)

    driver.switch_to.window(driver.window_handles[0])
    
    time.sleep(0.2)

    try:
        wait = WebDriverWait(driver, 5) 
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.prev_next_area > span > a.next_btn')))
        driver.find_element(By.CSS_SELECTOR, 'div.prev_next_area > span > a.next_btn').click()
    except:
        print("maybe skipping page: ", i)
        continue

    time.sleep(0.5)

pbar.close()

  0%|          | 0/11105 [00:00<?, ?it/s]

Error loading page:  about:blank
Error loading page:  https://www.dnn.de/politik/raketenbeschuss-auf-kramatorsk-putins-kampf-geht-weiter-XMGTBCBVEZD7LJ5QU6246JP7WQ.html
Error loading page:  about:blank
Error loading page:  about:blank
Error loading page:  about:blank
Error loading page:  about:blank
Error loading page:  about:blank
Error loading page:  about:blank
Error loading page:  about:blank
Error loading page:  about:blank
Error loading page:  about:blank
Error loading page:  about:blank
Error loading page:  https://www.dnn.de/politik/gerhard-schroeder-klagt-gegen-bundestag-gerichtsverhandlung-um-altkanzler-buero-5NMHVAI64VCWDG5I5WF33QOSGQ.html
Error loading page:  https://www.dnn.de/politik/russland-soll-laut-us-angaben-100000-mann-in-und-um-bachmut-verloren-haben-a011e7f3-35f2-4889-aa1a-49be4018ea06.html
Error loading page:  about:blank
Error loading page:  https://www.dnn.de/wirtschaft/hensoldt-ruestungsmanager-warnt-vor-putin-was-die-russen-gerade-lernen-GUD4ERGJZZHWBDZEN5GG2

In [6]:
df

,title,author,category,date,text
0,Erdogan: Wiederaufnahme des Getreideabkommens ...,NA,politik,"09.08.2023, 10:03 Uhr",Um eine Neuverhandlung des Getreideabkommens m...
1,Flug um den Mond: Crew der „Artemis 2“ besicht...,NA,wissen,"09.08.2023, 09:13 Uhr",Cape Canaveral. Die US-Raumfahrtbehörde Nasa s...
2,Kiew wirft Russland Angriff auf Rettungskräfte...,NA,politik,"09.08.2023, 06:37 Uhr",Kiew. Der ukrainische Präsident Wolodymyr Sele...
3,Russlands Ölexporte und griechische Schiffe: „...,Sven Christian Schulz,wirtschaft,"09.08.2023, 06:30 Uhr",Robin Brooks ist Chefökonom beim Institute of ...
4,Ukraine wirft Russland perfide Angriffe auf Hi...,NA,politik,"08.08.2023, 20:50 Uhr",Kiew. Die Ukraine hat Russland gezielte Angrif...
...,...,...,...,...,...
10113,Einschränkungen für Sender RT: Russischer Bots...,NA,politik,"02.01.2022, 12:59 Uhr","Berlin. Der russische Botschafter in Berlin, S..."
10114,Das Kalenderblatt am 2. Januar – was ist heute...,NA,panorama,"02.01.2022, 00:01 Uhr",Berlin. Das aktuelle Kalenderblatt für Sonntag...
10115,Zwischen Kriegsangst und neuen Machtgelüsten,Karl Doemens,politik,"01.01.2022, 15:51 Uhr",Es geht nicht nur um Russland. Aber die neben ...
10116,Auf der fünften Welle: Was das politische Jahr...,Eva Quadbeck,politik,"01.01.2022, 15:51 Uhr",Berlin. Es gibt diese charmante Komödie aus de...


In [8]:
df.to_csv(f"G:\coding\PcToLaptop\CER\project\data\\dnn_articles_2022_2024_1.csv",quoting=csv.QUOTE_NONNUMERIC)

In [7]:
len(not_readable) + len(df)

10858